# CDP Data 2022: Cleaning and Filtering

## Overview
This module preprocesses the raw CDP 2022 Climate Change dataset to create a manageable, filtered dataset focused on European electric utilities. The original CDP dataset exceeded Excel capacity limits, requiring Python-based processing for the 14 sample companies.

## Key Operations
- **Sector filtering**: Restricts data to "Electric utilities" sector and "Power generation" industry
- **Geographic filtering**: Limits scope to European countries for regional consistency  
- **Company filtering**: Extracts data for 14 predetermined sample companies
- **Structure optimization**: Removes unnecessary columns and metadata to reduce file size
- **Data standardization**: Sorts and formats data for subsequent extraction processes

## Output
Clean, filtered CDP dataset (`CDP_filtered_relevant_2023.xlsx`) containing only relevant sheets and companies for performance analysis. This serves as input for the extraction module.

## Note
This preprocessing step is essential due to the original dataset's size constraints and ensures data quality for the subsequent emission intensity calculations used in the greenwashing risk assessment.

In [ ]:
import pandas as pd
import os
from openpyxl import load_workbook



In [ ]:
import pandas as pd

# Define the file path
file_path = "data/CDP/Too_large_2023_Public_Climate_Change_Data_Aad.xlsx"

# Load all sheets from the Excel file
CDP_large = pd.read_excel(file_path, sheet_name=None)

# Add empty placeholder sheets if missing
if "How to use this dataset CC" not in CDP_large:
    CDP_large["How to use this dataset CC"] = pd.DataFrame()

if "Data orientation" not in CDP_large:
    CDP_large["Data orientation"] = pd.DataFrame()

# Define the list of sheets to keep (order matters!)
sheets_to_keep = [ 
    "How to use this dataset CC", "Data orientation", "Column Definitions", "Summary Data", "C0.1", "C0.2", "C0.3", 
    "C0.4", "C0.5", "C-EU0.7", "C2.1", "C2.1a", "C3.1", "C3.2", "C3.2a", "C3.2b", "C4.1", "C4.1a", "C4.1b", "C4.1c", 
    "C4.2", "C4.2a", "C4.2b", "C4.2c", "C4.3", "C4.3a", "C4.3b", "C4.3c", "C4.3d", "C-EU4.6", "C5.2", "C5.3", "C6.1", 
    "C6.2", "C6.3", "C6.4", "C6.4a", "C6.5", "C6.5a", "C6.7", "C6.7a", "C6.10", "C7.1", "C7.1a", "C7.2", "C7.3", "C7.3a", 
    "C7.3b", "C7.3c", "C7.5", "C7.6", "C7.6a", "C7.6b", "C7.6c", "C7.9", "C7.9a", "C7.9b", "C-EU7.1b", "C-EU8.2d", "C11.1", 
    "C11.1a", "C11.1b", "C11.1c", "C11.1d", "C11.2", "C11.2a", "C11.3", "C11.3a" 
]

# Build the filtered sheet dictionary in the correct order
CDP_filtered = {sheet: CDP_large[sheet] for sheet in sheets_to_keep if sheet in CDP_large}

# Save to new Excel
output_path = "data/2023_Public_Climate_Change_Data_Aad.xlsx"
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for sheet_name, df in CDP_filtered.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
# Define the file path
file_path = "data/2023_Public_Climate_Change_Data_Aad.xlsx"

# Load all sheet names
sheets = pd.ExcelFile(file_path).sheet_names

# Define sheets that should **not** skip any rows
sheets_no_skip = ["How to use this dataset", "Summary Data"]

# Read each sheet with conditional `skiprows`
CDP = {}
for sheet in sheets:
    if sheet in sheets_no_skip:
        CDP[sheet] = pd.read_excel(file_path, sheet_name=sheet)  # No skipping
    else:
        CDP[sheet] = pd.read_excel(file_path, sheet_name=sheet, skiprows=1)  # Skip first row

In [ ]:
# Get all sheet names
sheets = list(CDP.keys())

# Define sheets to process (from tab 4 onward)
sheets_to_filter = sheets[3:]  # Index 3 onwards (Python uses zero-based index)

# Define columns to filter
sector_col = "Primary questionnaire sector"
industry_col = "Primary industry"

# Apply filtering
for sheet in sheets_to_filter:
    df = CDP[sheet]

    # Ensure the required columns exist in the sheet before filtering
    if sector_col in df.columns and industry_col in df.columns:
        CDP[sheet] = df[
            (df[sector_col] == "Electric utilities") &
            (df[industry_col] == "Power generation")
        ]

In [ ]:
# Define the sheet and column to filter
sheet_name = "C0.3"
country_col = "Country/Area"

# List of European countries
european_countries = [
    "Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium",
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece",
    "Hungary", "Iceland", "Ireland", "Italy", "Kazakhstan", "Kosovo", "Latvia",
    "Liechtenstein", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco",
    "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal",
    "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain",
    "Sweden", "Switzerland", "Turkey", "Ukraine", "United Kingdom", "Vatican City",
    "United Kingdom of Great Britain and Northern Ireland"
]

# Apply country filter to all sheets except the first 3
for sheet in list(CDP.keys())[3:]:
    df = CDP[sheet]
    if country_col in df.columns:
        CDP[sheet] = df[df[country_col].isin(european_countries)]

# Print filtered data
CDP[sheet_name]

In [ ]:
# List of companies to keep
companies_to_keep = [
    "AKENERJİ ELEKTRİK ÜRETİM A.Ş.",
    "Arendals Fossekompani ASA",
    "Atlantica Sustainable Infrastructure PLC",
    "CEZ",
    "EDF",
    "EDP - Energias de Portugal S.A.",
    "Endesa",
    "ERG S.p.A",
    "Ørsted",
    "Polska Grupa Energetyczna (PGE) SA",
    "Romande Energie Holding SA",
    "Scatec ASA",
    "Solaria Energia y Medio Ambiente SA",
    "Terna Energy S.A"
]

# Apply company filter to all sheets from sheet 4 onwards
for sheet in list(CDP.keys())[3:]:  # From sheet 4 onwards
    df = CDP[sheet]
    
    # Ensure the 'Organization' column exists before filtering
    if 'Organization' in df.columns:
        CDP[sheet] = df[df['Organization'].isin(companies_to_keep)]

# Print filtered data for the last processed sheet
CDP[sheet]

In [ ]:
# Apply column dropping and sorting to sheets from index 4 onward
for i, sheet in enumerate(list(CDP.keys())[4:], start=4):
    df = CDP[sheet]

    # Reset index to ensure accurate column positions
    df.reset_index(drop=True, inplace=True)

    # Determine which columns to drop: first column and columns 3 to 15 (by index)
    cols_to_drop = []
    if df.shape[1] >= 9:
        cols_to_drop = [df.columns[0]] + list(df.columns[2:15])
    elif df.shape[1] >= 1:
        cols_to_drop = [df.columns[0]]

    df = df.drop(columns=cols_to_drop, errors="ignore")

    # Sort rows by 'Organization' if the column exists
    if 'Organization' in df.columns:
        df = df.sort_values(by='Organization', ascending=True, na_position='last')

    CDP[sheet] = df


In [ ]:
# Define the Excel output path
excel_output_path = "data/CDP_filtered_relevant_2023.xlsx"

# Save the filtered data to a new Excel file
with pd.ExcelWriter(excel_output_path, engine="openpyxl") as writer:
    for sheet_name, df in CDP.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
# Auto-adjust column width based on the longest text in the first row
wb = load_workbook(excel_output_path)
for sheet in wb.sheetnames:
    ws = wb[sheet]
    for col in ws.columns:
        max_length = 0
        col_letter = col[0].column_letter  # Get the column letter
        for cell in col[:1]:  # Only check the first row
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        adjusted_width = max_length + 3  # Add padding for readability
        ws.column_dimensions[col_letter].width = adjusted_width

# Save the modified Excel file
wb.save(excel_output_path)

In [ ]:
# Load CDP Excel file and specific sheet
CDP = pd.read_excel("data/CDP_filtered_relevant_2023.xlsx", sheet_name=None)  # Load all sheets

CDP

In [ ]:
# Get all sheet names
sheet_names = list(CDP.keys())

import re

# Shorten column names from sheet 5 onward (index 4)
for sheet in sheet_names[4:]:
    df = CDP[sheet]

    new_columns = []
    for i, col in enumerate(df.columns):
        if i <= 3:
            new_columns.append(col)
        else:
            col_str = str(col)

            # Split by underscores to get code parts (e.g., C4.1a_C2)
            code_parts = col_str.split("_")
            if len(code_parts) >= 2:
                prefix = f"{code_parts[0]}_{code_parts[1]}"
            else:
                prefix = code_parts[0]

            # Get the label after the last ' - '
            if " - " in col_str:
                label = col_str.split(" - ")[-1].strip()
                new_col = f"{prefix} - {label}"
            else:
                new_col = col_str  # fallback

            new_columns.append(new_col)

    df.columns = new_columns
    CDP[sheet] = df


# Save updated Excel
excel_output_path = "data/CDP/CDP_2023_final_test.xlsx"
with pd.ExcelWriter(excel_output_path, engine="openpyxl") as writer:
    for sheet_name, df in CDP.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
from openpyxl.utils import get_column_letter

# Auto-adjust column widths
wb = load_workbook(excel_output_path)
for sheet in wb.sheetnames:
    ws = wb[sheet]
    for col in ws.columns:
        max_length = 0
        col_letter = get_column_letter(col[0].column)
        for cell in col[:1]:  # Only check first row
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[col_letter].width = max_length + 3  # padding

wb.save(excel_output_path)

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter

# Define fill color
grey_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")

# Loop over sheets from tab 5 onward
for sheet_name in wb.sheetnames[4:]:
    ws = wb[sheet_name]

    # === STEP 1: Delete rows where every column from D onward = "Question not applicable" ===
    rows_to_delete = []
    for row in range(2, ws.max_row + 1):  # Skip header
        values = [ws.cell(row=row, column=col).value for col in range(4, ws.max_column + 1)]
        if all(str(v).strip() == "Question not applicable" for v in values):
            rows_to_delete.append(row)

    # Delete rows from bottom to top to avoid index shift
    for row in reversed(rows_to_delete):
        ws.delete_rows(row)

    # === STEP 2: Alternate row colors by company ===
    prev_company = None
    use_grey = False
    for row in range(2, ws.max_row + 1):
        current_company = ws[f"A{row}"].value
        if current_company != prev_company:
            use_grey = not use_grey
            prev_company = current_company

        if use_grey:
            for col in range(1, ws.max_column + 1):
                ws.cell(row=row, column=col).fill = grey_fill

# Save the final cleaned and formatted workbook
wb.save("data/CDP/CDP_2023_short_color_relevant_rows.xlsx")
